<a href="https://colab.research.google.com/github/tomonari-masada/course2022-stats2/blob/main/05_divergent_transitions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5. サンプリングがうまくいかない場合の例
* 密度関数がhigh curvatureであるとき、サンプリングが上手くいかないことがある。
 * high curvatureになっている部分が、サンプリングによって全くカバーされなかったりする。
* この場合、reparameterizationを使うと、問題が解決することがある。
 * https://mc-stan.org/docs/stan-users-guide/reparameterization.html

## 5.0 準備

In [ ]:
import warnings

import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pymc3 as pm

warnings.simplefilter(action="ignore", category=FutureWarning)

%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")

SEED = 20220307

## 5.1 サンプリングがうまく行かない理由
* 参考資料
 * https://mc-stan.org/docs/reference-manual/divergent-transitions.html
* HMCは、勾配を使ってfirst-orderの近似を行っているため、high curvatureな場所では、本当に求めたいものから大きく外れてしまうことがある。すると、サンプリングがパラメータ空間内での単なるランダムウォークに近くなってしまい、密度関数の"濃淡"を反映しないものになってしまう。
 * 別の参考資料 https://norimune.net/3149

## 5.2 例題
* 下のような確率分布を考える( https://mc-stan.org/docs/stan-users-guide/reparameterization.html )。


$$\begin{align} 
y & \sim N(0, 3^2) \\
x_i & \sim N(0, e^y), \text{ $i=1,\ldots, 9$ }  
\end{align}$$


* 同時分布を式で書くと・・・
$$p(y, x_1, \ldots, x_9) = p(y) \prod_{i=1}^9 p(x_i | y)$$
where
$$ p(y) = \frac{1}{\sqrt{2\pi 3^2}} \exp\bigg( - \frac{y^2}{2 \times 3^2}\bigg)  $$
and
$$ p(x_i | y) = \frac{1}{\sqrt{2\pi e^y}} \exp\bigg( - \frac{x_i^2}{2e^y} \bigg) $$

* この分布の問題点
 * $y$の値が小さいとき、$x_i$の従う正規分布が、非常にpeakyな密度関数を持つ
 * 従って、サンプリングが困難になる。

* 今回は、この分布からサンプリングする（＝この分布に従う乱数を発生させる）。
 * つまり、今日はベイズの話をするのではなく（＝事後分布からのサンプリングをするのではなく）・・・
 * NUTSサンプラーでもうまくいかない場合がどんな場合かを、単に説明する。

### 5.2.1 実装方法 (1)
* これは悪い実装方法。
 * divergent transitionが発生する。
 * さらに、周辺分布$p(y)$は正規分布となるはずなのに、ヒストグラムが全く正規分布の形にならない。

In [ ]:
with pm.Model() as model:
  y = pm.Normal("y", mu=0, sd=3)
  x = pm.Normal("x", mu=0, sd=(y/2).exp(), shape=9)

* 　今回は、`return_inferencedata=True`と設定せず、arviz向けではない形式、元のPyMC3の形式で、サンプリング結果を得る
 * arviz向けのサンプリング結果で同じようにdivergenceの分析を行う方法は後で説明する。

In [ ]:
with model:
  trace = pm.sample(2000, cores=4, random_seed=SEED)

In [ ]:
type(trace)

In [ ]:
trace['x'].shape

In [ ]:
x0 = trace['x'][:,0]

In [ ]:
x0.shape

In [ ]:
y = trace['y']

In [ ]:
y.shape

* divergenceとは

> "A divergence arises when the simulated Hamiltonian trajectory departs from the true trajectory as measured by departure of the Hamiltonian value from its initial value. When this divergence is too high, the simulation has gone off the rails and cannot be trusted. The positions along the simulated trajectory after the Hamiltonian diverges will never be selected as the next draw of the MCMC algorithm, potentially reducing Hamiltonian Monte Carlo to a simple random walk and biasing estimates by not being able to thoroughly explore the posterior distribution." ( https://mc-stan.org/docs/reference-manual/divergent-transitions.html )



* PyMC3でのdivergenceのチェック方法

In [ ]:
divergent = trace['diverging']

In [ ]:
divergent.shape

In [ ]:
divergent

In [ ]:
divergent.sum()

In [ ]:
np.where(divergent)

* chainの中でdivergenceが発生したサンプルを赤で示してみる。

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(x0[~ divergent], y[~ divergent], color='g')
plt.scatter(x0[divergent], y[divergent], color='r')
plt.axis([-20, 20, -9, 9])
plt.ylabel('y')
plt.xlabel('x[0]')
plt.title('scatter plot between y and x[0]');

* 上のプロットで分かるように、図の下の方の領域で全くサンプルが取られていない。

* $y$のヒストグラムを確認する。
 * yの周辺分布（下の式）は正規分布になるはずだが・・・
$$p(y) = \idotsint p(y, x_1, \ldots, x_9) dx_1 \cdots dx_9$$

In [ ]:
sns.displot(y, kind="kde");

### 5.2.2 実装方法 (2)
* これは、reparameterizationを使うことで改良された実装。
 * divergent transitionは起こらない。
 * $y$のサンプルのヒストグラムも正規分布の形を示す。

* 元のモデルは
$$\begin{align} 
y & \sim N(0, 3^2) \\
x_i & \sim N(0, e^y), \text{ $i=1,\ldots, 9$ }  
\end{align}$$


* $x_i$を$N(0, e^y)$からサンプリングする、という実装をやめて、代わりに
 * まず$x_\text{raw}$を$N(0,1)$からサンプリングし・・・
 * その$x_\text{raw}$を$x = e^{y/2} x_\text{raw}$という式で変換している。

In [ ]:
with pm.Model() as model_revised:
  y = pm.Normal("y", mu=0, sd=3)
  x_raw = pm.Normal("x_raw", mu=0, sd=1, shape=9)
  x = pm.Deterministic("x", (y/2).exp() * x_raw)

In [ ]:
with model_revised:
  trace_revised = pm.sample(2000, cores=4, random_seed=SEED)

In [ ]:
x0 = trace_revised['x'][:, 0]
y = trace_revised['y']
divergent = trace_revised['diverging']

In [ ]:
divergent.sum()

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(x0[~ divergent], y[~ divergent], color='g')
plt.scatter(x0[divergent], y[divergent], color='r')
plt.axis([-20, 20, -9, 9])
plt.ylabel('y')
plt.xlabel('x[0]')
plt.title('scatter plot between y and x[0]');

In [ ]:
sns.displot(y, kind="kde");

## 5.3 The Eight Schools Model
* 下記Webページにある「A Centered Eight Schools Implementation」の実験を再現してみる。
 * https://docs.pymc.io/en/v3/pymc-examples/examples/diagnostics_and_criticism/Diagnosing_biased_Inference_with_Divergences.html




> "Hamiltonian Monte Carlo, for example, is especially powerful in this regard as its failures to be geometrically ergodic with respect to any target distribution manifest in distinct behaviors that have been developed into sensitive diagnostics. One of these behaviors is the appearance of divergences that indicate the Hamiltonian Markov chain has encountered regions of high curvature in the target distribution which it cannot adequately explore."



* arviz向けにコードを書き直した。

### 5.5.1 データセット
* `y`が観測データ
 * 各校で同じコーチングを実施し、その前後で学力テストの点数がどう変化したかを表す。
 * 正確には、この`y`は観測データではなく、8つの学校ごとに別々の回帰分析によって得られたestimates。
 * cf. https://arxiv.org/abs/1507.04544 のSection 4.1
* `sigma`が既知のパラメータ
 * これも、8つの学校ごとに別々の回帰分析によって得られたstandard errors。

In [ ]:
# Data of the Eight Schools Model
y = np.asarray([28,  8, -3,  7, -1,  1, 18, 12], dtype=float)
sigma = np.asarray([15, 10, 16, 11,  9, 11, 10, 18], dtype=float)
J = y.shape[0]

* このデータを以下のようにモデリングする。
$$\begin{align}
\mu & \sim N(0,5) \notag \\
\tau & \sim \text{Half-Cauchy}(5) \\
\theta_n & \sim N(\mu, \tau) \\
y_n & \sim N(\theta_n, \sigma_n^2)
\end{align}$$
 * $y_n$が上記コードの`y`に対応する。
 * $\sigma_n$が上記コードの`sigma`に対応する。

* Half-cauchy分布については下記ページを参照
 * https://distribution-explorer.github.io/continuous/halfcauchy.html
> "The Half-Cauchy distribution with 𝜇=0 is a useful prior for nonnegative parameters that may be very large, as allowed by the very heavy tails of the Half-Cauchy distribution."

### 5.5.2 実装方法(1)
* これは良くない実装方法。
 * 後でreparameterizationを使って改良する。

In [ ]:
with pm.Model() as Centered_eight:
  mu = pm.Normal('mu', mu=0, sd=5)
  tau = pm.HalfCauchy('tau', beta=5)
  theta = pm.Normal('theta', mu=mu, sd=tau, shape=J)
  obs = pm.Normal('obs', mu=theta, sd=sigma, observed=y)

* `return_inferencedata=True`としてarviz向けのサンプリング結果データを得る。　

In [ ]:
with Centered_eight:
  trace = pm.sample(2000, cores=4, random_seed=SEED, return_inferencedata=True)

In [ ]:
az.summary(trace)

* `r_hat`が1.0から離れているものもある。
* `tau`に問題がありそう。

In [ ]:
pm.traceplot(trace, var_names=['tau']);

* このデータについては$\tau$の"真の値"が分かっているらしい。
 * "真の値"の詳細は https://discourse.pymc.io/t/how-is-the-true-value-of-tau-in-the-eight-schools-model-known/1932
* logスケールでプロットして、真の値からのズレを見てみる。
 * $\log \tau$の真の値は0.7657852らしいです。

In [ ]:
logtau = np.log(trace.posterior['tau'].data)
plt.figure(figsize=(15, 4))
plt.axhline(0.7657852, lw=2.5, color='gray')
for j in range(logtau.shape[0]):
  mlogtau = [np.mean(logtau[j,:i]) for i in np.arange(1, len(logtau[j]))]
  plt.plot(mlogtau, lw=2.5)
plt.ylim(0, 2)
plt.xlabel('Iteration')
plt.ylabel('MCMC mean of log(tau)')
plt.title('MCMC estimation of log(tau)');

* arviz向けのデータの場合、divergencesの情報がどこに保存されているか、調べる。

In [ ]:
trace.sample_stats

In [ ]:
trace.sample_stats.diverging

In [ ]:
chain_id = 0
divergent = trace.sample_stats.diverging.data[chain_id]

In [ ]:
theta_trace = trace.posterior['theta'][chain_id]
theta0 = theta_trace[:,0]
plt.figure(figsize=(10, 6))
plt.scatter(theta0[~divergent], logtau[chain_id][~divergent], color='g')
plt.scatter(theta0[divergent], logtau[chain_id][divergent], color='r')
plt.axis([-20, 50, -6, 4])
plt.ylabel('log(tau)')
plt.xlabel('theta[0]')
plt.title('scatter plot between log(tau) and theta[0]');

In [ ]:
chain_id = 1
divergent = trace.sample_stats.diverging.data[chain_id]
theta_trace = trace.posterior['theta'][chain_id]
theta0 = theta_trace[:,0]
plt.figure(figsize=(10, 6))
plt.scatter(theta0[~divergent], logtau[chain_id][~divergent], color='g')
plt.scatter(theta0[divergent], logtau[chain_id][divergent], color='r')
plt.axis([-20, 50, -6, 4])
plt.ylabel('log(tau)')
plt.xlabel('theta[0]')
plt.title('scatter plot between log(tau) and theta[0]');

### 5.5.3 実装方法(2)
* reparameterizationを使う。

* $\theta_i$を$N(\mu, \tau)$からサンプリングする、という実装をやめて、代わりに
 * まず$\tilde{\theta}_i$を$N(0,1)$からサンプリングし・・・
 * その$\tilde{\theta}_i$を$\theta_i = \mu + \tau \tilde{\theta}_i$という式で変換している。

In [ ]:
with pm.Model() as NonCentered_eight:
  mu = pm.Normal('mu', mu=0, sd=5)
  tau = pm.HalfCauchy('tau', beta=5)
  theta_tilde = pm.Normal('theta_t', mu=0, sd=1, shape=J)
  theta = pm.Deterministic('theta', mu + tau * theta_tilde) # ここでreparameterizationを使用
  obs = pm.Normal('obs', mu=theta, sd=sigma, observed=y)

In [ ]:
with NonCentered_eight:
  trace = pm.sample(2000, cores=4, random_seed=SEED, return_inferencedata=True)

In [ ]:
az.summary(trace)

In [ ]:
pm.traceplot(trace, var_names=['tau']);

In [ ]:
logtau = np.log(trace.posterior['tau'].data)
plt.figure(figsize=(15, 4))
plt.axhline(0.7657852, lw=2.5, color='gray')
for j in range(logtau.shape[0]):
  mlogtau = [np.mean(logtau[j,:i]) for i in np.arange(1, len(logtau[j]))]
  plt.plot(mlogtau, lw=2.5)
plt.ylim(0, 2)
plt.xlabel('Iteration')
plt.ylabel('MCMC mean of log(tau)')
plt.title('MCMC estimation of log(tau)');

In [ ]:
chain_id = 0
divergent = trace.sample_stats.diverging.data[chain_id]
theta_trace = trace.posterior['theta'][chain_id]
theta0 = theta_trace[:,0]
plt.figure(figsize=(10, 6))
plt.scatter(theta0[~divergent], logtau[chain_id][~divergent], color='g')
plt.scatter(theta0[divergent], logtau[chain_id][divergent], color='r')
plt.axis([-20, 50, -6, 4])
plt.ylabel('log(tau)')
plt.xlabel('theta[0]')
plt.title('scatter plot between log(tau) and theta[0]');

In [ ]:
chain_id = 1
divergent = trace.sample_stats.diverging.data[chain_id]
theta_trace = trace.posterior['theta'][chain_id]
theta0 = theta_trace[:,0]
plt.figure(figsize=(10, 6))
plt.scatter(theta0[~divergent], logtau[chain_id][~divergent], color='g')
plt.scatter(theta0[divergent], logtau[chain_id][divergent], color='r')
plt.axis([-20, 50, -6, 4])
plt.ylabel('log(tau)')
plt.xlabel('theta[0]')
plt.title('scatter plot between log(tau) and theta[0]');

### 5.5.4 採択率(acceptance rate)を調整する

> "target_accept : float in [0, 1]. The step size is tuned such that we approximate this acceptance rate. Higher values like 0.9 or 0.95 often work better for problematic posteriors." (https://docs.pymc.io/api/inference.html ) 


In [ ]:
with NonCentered_eight:
  step = pm.NUTS(target_accept=.90)
  trace = pm.sample(2000, step=step, cores=4, random_seed=SEED, return_inferencedata=True)

In [ ]:
with NonCentered_eight:
  step = pm.NUTS(target_accept=.95)
  trace = pm.sample(2000, step=step, cores=4, random_seed=SEED, return_inferencedata=True)

In [ ]:
logtau = np.log(trace.posterior['tau'].data)
plt.figure(figsize=(15, 4))
plt.axhline(0.7657852, lw=2.5, color='gray')
for j in range(logtau.shape[0]):
  mlogtau = [np.mean(logtau[j,:i]) for i in np.arange(1, len(logtau[j]))]
  plt.plot(mlogtau, lw=2.5)
plt.ylim(0, 2)
plt.xlabel('Iteration')
plt.ylabel('MCMC mean of log(tau)')
plt.title('MCMC estimation of log(tau)');